In [ ]:
# backend/notebooks/mercado_publico_api_exploration.ipynb

# %% [markdown]
# # Exploración API Mercado Público
# 
# Este notebook explora la API de Mercado Público para el módulo de licitaciones.

# %%
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

# %% [markdown]
# ## Configuración inicial

# %%
BASE_URL = "https://api.mercadopublico.cl"
TODAY = datetime.now().strftime("%Y-%m-%d")
LAST_WEEK = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

# Tu API Key (debes obtenerla de Mercado Público)
API_KEY = "tu_api_key_aqui"

# %% [markdown]
# ## Funciones auxiliares

# %%
def get_licitaciones(fecha_desde, fecha_hasta, estado="Publicada", codigo_region=None):
    """
    Obtiene licitaciones según parámetros de búsqueda
    """
    endpoint = f"{BASE_URL}/servicios/v1/publico/licitaciones.json"
    
    params = {
        "fechaDesde": fecha_desde,
        "fechaHasta": fecha_hasta,
        "estado": estado,
        "ticket": API_KEY
    }
    
    if codigo_region:
        params["codigoRegion"] = codigo_region
    
    response = requests.get(endpoint, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# %% [markdown]
# ## Prueba de obtención de licitaciones

# %%
# Obtener licitaciones de la última semana
licitaciones = get_licitaciones(LAST_WEEK, TODAY)

# Mostrar algunas licitaciones
if licitaciones:
    print(f"Total licitaciones: {len(licitaciones['Listado'])}")
    print("\nEjemplo de licitación:")
    print(json.dumps(licitaciones['Listado'][0], indent=2))

# %% [markdown]
# ## Análisis de datos

# %%
if licitaciones:
    # Convertir a DataFrame para análisis
    df = pd.DataFrame(licitaciones['Listado'])
    
    # Mostrar resumen
    print("\nResumen de licitaciones:")
    print(df[['CodigoExterno', 'Nombre', 'FechaCierre', 'Comprador']].head())
    
    # Estadísticas básicas
    print("\nEstadísticas:")
    print(f"Licitaciones por región:\n{df['CodigoRegion'].value_counts()}")

In [ ]:
PALABRAS_CLAVE = [
    "computador", "notebook", "hardware", "mantención informática",
    "tecnico computación", "monitor", "mouse", "teclado",
    "servidor", "impresora", "redes", "componentes electrónicos"
]

CODIGOS_CATEGORIA = [
    "09000000",  # Equipos y componentes informáticos (código Mercado Público)
    "32000000",  # Componentes electrónicos
    "72000000"   # Servicios TI y mantenimiento
]

In [ ]:
# backend/notebooks/mercado_publico_api_exploration.ipynb

# Añadir esta celda para probar los filtros específicos
# %%
from backend.services.licitaciones_service import LicitacionesService

service = LicitacionesService(api_key=API_KEY)

licitaciones_relevantes = service.obtener_licitaciones(
    palabra_clave="|".join(PALABRAS_CLAVE),  # Búsqueda OR
    codigo_categoria=",".join(CODIGOS_CATEGORIA),  # Filtro múltiple
    limit=50
)

print(f"Licitaciones encontradas: {len(licitaciones_relevantes)}")